In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import json

sys.path.append("/Users/ruhana/Agent-E/test")
from load_annotations import AnnotationLoader

None


In [2]:
LOG_PATH = "/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text_vision/"
RESULT_PATH = "/Users/ruhana/Agent-E/new_ruhana_notes/All/results/results_for_test_full_text_vision/"
self_refine_annotations = AnnotationLoader(LOG_PATH, RESULT_PATH)

# Load final results as a dataframe
self_refine_results = self_refine_annotations.get_all_annotation_results()
self_refine_results[:2]

[Errno 2] No such file or directory: '/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text_vision/logs_for_task_382/execution_logs_382.json'
[Errno 2] No such file or directory: '/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text_vision/logs_for_task_556/execution_logs_556.json'


,task_id,task_index,start_url,intent,last_url,tct,start_ts,completion_ts,last_statement,validator_last_statement,score,reason,validate_score,validate_reason,screenshot_rate,validation_count,total_main_chat_token,main_chat_length,total_trajectory_length
0,0,0,https://www.allrecipes.com/,Provide a recipe for vegetarian lasagna with m...,https://www.allrecipes.com/recipe/22729/spinac...,153.790992,2024-08-22 12:54:40,2024-08-22 12:57:15,Here is a highly rated vegetarian lasagna reci...,The task was completed. The workflow was succe...,1.0,None,1.0,None,4/4,1,1301,14,12
0,1,1,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna that ha...,https://www.allrecipes.com/recipe/236878/debbi...,83.462629,2024-09-05 11:29:05,2024-09-05 11:30:30,I found a recipe for a vegetarian lasagna that...,The task was completed. The workflow was succe...,1.0,None,1.0,None,4/4,1,610,11,11


In [3]:
all_self_refinement_results = {}

for mode in ["text", "vision", "text_vision"]:
    LOG_PATH = f"/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_{mode}/"
    RESULT_PATH = f"/Users/ruhana/Agent-E/new_ruhana_notes/All/results/results_for_test_full_{mode}/"
    annon_loader = AnnotationLoader(LOG_PATH, RESULT_PATH)
    all_self_refinement_results[mode] = annon_loader.get_all_annotation_results()
    print(f"Loaded {len(all_self_refinement_results[mode])} items")

refiner_text = all_self_refinement_results['text']
refiner_vision = all_self_refinement_results['vision']
refiner_hybrid = all_self_refinement_results['text_vision']

print(f"Agent-E Text Acc: {refiner_text['score'].mean()}")
print(f"Agent-E Vision Acc: {refiner_vision['score'].mean()}")
print(f"Agent-E Hybrid Acc: {refiner_hybrid['score'].mean()}")

Loaded 392 items
Loaded 455 items
[Errno 2] No such file or directory: '/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text_vision/logs_for_task_382/execution_logs_382.json'
[Errno 2] No such file or directory: '/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text_vision/logs_for_task_556/execution_logs_556.json'
Loaded 471 items
Agent-E Text Acc: 0.8137755102040817
Agent-E Vision Acc: 0.8347252747252748
Agent-E Hybrid Acc: 0.8021231422505308


In [4]:
LOG_PATH = "/Users/ruhana/Agent-E/ruhana_notes/baseline_annotated/original_annotations/All/logs"
RESULT_PATH = "/Users/ruhana/Agent-E/ruhana_notes/baseline_annotated/original_annotations/All/results"
original_annotation = AnnotationLoader(LOG_PATH, RESULT_PATH)

# Load final results as a dataframe
original_results = original_annotation.get_all_annotation_results()
original_results.columns

# make sure original matches the same subset!!!
self_refine_task_id = list(self_refine_results['task_id'])
original_results = original_results[original_results['task_id'].isin(self_refine_task_id)]

In [5]:
# Helper function to load final_responses from original run
def get_final_response(annotation_loader, _task_id):
    raw_final_response = annotation_loader.get_high_level_trajectory(_task_id)[-1]
    try:
        temp_json = json.loads(raw_final_response.get('content'))
        _final_response = temp_json['final_response']
        return _final_response
    except Exception as e:
        return "" # no final_response

# Get all last responses
original_results['last_statement'] = original_results['task_id'].apply(
    lambda x: get_final_response(annotation_loader=original_annotation, _task_id=x)
)
original_results[:2]

,task_id,task_index,start_url,intent,last_url,tct,start_ts,completion_ts,last_statement,score,reason,compute_cost.cost,compute_cost.prompt_tokens,compute_cost.completion_tokens,compute_cost.total_tokens,total_main_chat_token,main_chat_length,total_trajectory_length
0,0,0,https://www.allrecipes.com/,Provide a recipe for vegetarian lasagna with m...,https://www.allrecipes.com/recipe/21090/vegeta...,145.615960,2024-06-18 15:49:48,2024-06-18 15:52:16,Here is a recipe for Vegetarian Four Cheese La...,1.0,None,0.19763,17390.0,791.0,18181.0,1161,14,15
0,1,1,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna that ha...,https://www.allrecipes.com/recipe/78358/easy-v...,263.331766,2024-06-18 15:55:14,2024-06-18 15:59:38,After reviewing several recipes for vegetarian...,0.0,All these recipes do not explicitly include zu...,0.56686,53926.0,920.0,54846.0,1229,36,41


In [6]:
# Get the subset of tasks that were improve via self-refinement
original_wrong = set(original_results[original_results['score'] == 0.0]['task_id'])
original_correct = set(original_results[original_results['score'] == 1.0]['task_id'])

# Get improvement set for all 
refiner_correct_text = set(refiner_text[refiner_text['score'] == 1.0]['task_id'])
refiner_correct_vision = set(refiner_vision[refiner_vision['score'] == 1.0]['task_id'])
refiner_correct_hybrid = set(refiner_hybrid[refiner_hybrid['score'] == 1.0]['task_id'])

improved_tasks_text = list(original_wrong - refiner_correct_text)
improved_tasks_vision = list(original_wrong - refiner_correct_vision)
improved_tasks_hybrid = list(original_wrong - refiner_correct_hybrid)

print(len(improved_tasks_text))
print(len(improved_tasks_vision))
print(len(improved_tasks_hybrid))

needs_labeling_for_error_analysis = set(improved_tasks_text + improved_tasks_vision + improved_tasks_hybrid)
needs_labeleing_df = original_results[original_results['task_id'].isin(needs_labeling_for_error_analysis)].sort_values(by='task_id', ascending=True).reset_index(drop=True)
print(len(needs_labeleing_df))
# # Warning: Use use this if you are trying to extract the subset of annotate
# # Build a table of items that you need to label to categorize manually
# needs_labeleing_df = needs_labeleing_df.merge(self_aware_original, on='task_id', how='left')
# needs_labeleing_df[['task_id', 'task_index', 'start_url', 'intent', 'last_url', 'tct', 'last_statement', 'score', 'reason','Failure Mode']]
# needs_labeleing_df.rename({"Failure Mode":"self_aware_failure"})

# # Join with annotation of self-ware errors
# self_aware_original_path = "/Users/ruhana/Agent-E/ruhana_notes/baseline_annotated/original_annotations/original_annotation_error_analysis.csv"
# self_aware_original = pd.read_csv(self_aware_original_path)[['task_id', 'Failure Mode']]
needs_labeleing_df[:2]

52
38
39
70


,task_id,task_index,start_url,intent,last_url,tct,start_ts,completion_ts,last_statement,score,reason,compute_cost.cost,compute_cost.prompt_tokens,compute_cost.completion_tokens,compute_cost.total_tokens,total_main_chat_token,main_chat_length,total_trajectory_length
0,6,6,https://www.allrecipes.com/,Search for a vegetarian lasagna recipe that ha...,https://www.allrecipes.com/search?q=vegetarian...,94.735328,2024-06-18 16:55:50,2024-06-18 16:57:25,After reviewing the search results on AllRecip...,0.0,None of the recipes listed in the first ten re...,0.13249,11266.0,661.0,11927.0,773,10,10
1,16,16,https://www.allrecipes.com/,Find a five-star rated chocolate chip cookie r...,https://www.allrecipes.com/recipe/10813/best-c...,113.023372,2024-06-18 19:09:50,2024-06-18 19:11:45,"The 'Best Chocolate Chip Cookies' recipe, whic...",0.0,User has asked for a five-star ratings recipe ...,0.11940,10710.0,410.0,11120.0,547,10,12


## Analyzing Failure Cases By Manual Annotations

In [45]:
annotated_data_file = "/Users/ruhana/Agent-E/notebooks/error_analysis_outputs/original_annotations_error_analysis.csv"
error_annotated_df = pd.read_csv(annotated_data_file)

# remove unnamed columns caused from excel sheet
error_annotated_df = error_annotated_df[error_annotated_df.filter(regex='^(?!Unnamed)').columns]

# Merge original dataframe with the dataframe with annotations
original_results = original_results[['task_id', 'task_index', 'start_url', 'intent', 'last_url', 'tct',
       'start_ts', 'completion_ts', 'last_statement', 'score', 'reason',
       'compute_cost.cost', 'compute_cost.prompt_tokens',
       'compute_cost.completion_tokens', 'compute_cost.total_tokens',
       'total_main_chat_token', 'main_chat_length', 'total_trajectory_length']]
error_annotated_df_short = error_annotated_df[['task_id', 'self_aware_failure', 'error_category', 'error_category_reason']]
original_results = original_results.merge(error_annotated_df_short, on='task_id', how='left')

# Peak at the original dataframe
original_results_short = original_results[['task_id', 'intent', 'score', 'reason', 'self_aware_failure', 'error_category', 'error_category_reason']]
original_results_short[:10]

,task_id,intent,score,reason,self_aware_failure,error_category,error_category_reason
0,0,Provide a recipe for vegetarian lasagna with more than 100 reviews and a rating of at least 4.5 stars suitable for 6 people.,1.0,None,NaN,NaN,NaN
1,1,Find a recipe for a vegetarian lasagna that has at least a four-star rating and uses zucchini.,0.0,All these recipes do not explicitly include zucchini in their ingredients list.,NaN,NaN,NaN
2,2,Find a recipe for a vegetarian lasagna under 600 calories per serving that has a prep time of less than 1 hour.,0.0,"After reviewing the first three vegetarian lasagna recipes from Allrecipes, none meet the specific criteria of being under 600 calories per serving with a prep time of less than 1 hour.",NaN,NaN,NaN
3,3,Locate a recipe for vegan chocolate chip cookies with over 60 reviews and a rating of at least 4.5 stars on Allrecipes.,1.0,None,NaN,NaN,NaN
4,4,Find a recipe for Baked Salmon that takes less than 30 minutes to prepare and has at least a 4 star rating based on user reviews.,1.0,None,NaN,NaN,NaN
5,6,Search for a vegetarian lasagna recipe that has at least a four-star rating and over 500 reviews.,0.0,None of the recipes listed in the first ten results meet the criteria of having at least a four-star rating and over 500 reviews.,Aware,poor_navigation,NaN
6,8,Search for a recipe for Beef Wellington on Allrecipes that has at least 200 reviews and an average rating of 4.5 stars or higher. List the main ingredients required for the dish.,0.0,none of the recipes meet the specified criteria of having at least 200 reviews and an average rating of 4.5 stars or higher.,NaN,NaN,NaN
7,9,"Find a high-rated recipe for vegetarian lasagna, list the key ingredients required, and include the total preparation and cook time stated on the recipe.",1.0,None,NaN,NaN,NaN
8,10,"Find The Most Popular Recipes of the 1960s, noting the recipe name, preparation time and total time of the second recipe in this collection.",1.0,None,NaN,NaN,NaN
9,11,Discover a suitable chocolate cupcake recipe on Allrecipes that has a preparation time of under 1 hour and at least 100 user reviews.,1.0,None,NaN,NaN,NaN


In [51]:
# break down by category 
improved_tasks_text = list(original_wrong - refiner_correct_text)
improved_tasks_vision = list(original_wrong - refiner_correct_vision)
improved_tasks_hybrid = list(original_wrong - refiner_correct_hybrid)

improved_by_text_df = original_results[original_results['task_id'].isin(improved_tasks_text)].sort_values(by='task_id', ascending=True).reset_index(drop=True)
improved_by_vision_df = original_results[original_results['task_id'].isin(improved_tasks_vision)].sort_values(by='task_id', ascending=True).reset_index(drop=True)
improved_by_hybrid_df = original_results[original_results['task_id'].isin(improved_tasks_hybrid)].sort_values(by='task_id', ascending=True).reset_index(drop=True)

print(f"Self-Refiner (Text) Improved {len(improved_by_text_df)} tasks")
print("Breakdown By Category:\n", improved_by_text_df['error_category'].value_counts(normalize=True))

print(f"\nSelf-Refiner (Vision) Improved {len(improved_by_vision_df)} tasks")
print("Breakdown By Category:\n", improved_by_vision_df['error_category'].value_counts(normalize=True))

print(f"\nSelf-Refiner (Hybrid) Improved {(len(improved_by_hybrid_df))} tasks")
print("Breakdown By Category:\n", improved_by_hybrid_df['error_category'].value_counts(normalize=True))

Self-Refiner (Text) Improved 52 tasks
Breakdown By Category:
 error_category
missing_skill             0.403846
poor_navigation           0.250000
other                     0.134615
dom_interpretation        0.134615
incomplete_answer         0.057692
hallucinationed answer    0.019231
Name: proportion, dtype: float64

Self-Refiner (Vision) Improved 38 tasks
Breakdown By Category:
 error_category
missing_skill             0.473684
poor_navigation           0.236842
incomplete_answer         0.078947
other                     0.078947
dom_interpretation        0.078947
hallucinationed answer    0.052632
Name: proportion, dtype: float64

Self-Refiner (Hybrid) Improved 39 tasks
Breakdown By Category:
 error_category
missing_skill         0.512821
poor_navigation       0.282051
incomplete_answer     0.102564
other                 0.051282
dom_interpretation    0.051282
Name: proportion, dtype: float64


In [50]:
print(f"Self-Refiner (Text) Improved {len(improved_by_text_df)} tasks")
print("Breakdown By Category:\n", improved_by_text_df['self_aware_failure'].value_counts())

print(f"\nSelf-Refiner (Vision) Improved {len(improved_by_vision_df)} tasks")
print("Breakdown By Category:\n", improved_by_vision_df['self_aware_failure'].value_counts())

print(f"\nSelf-Refiner (Hybrid) Improved {(len(improved_by_hybrid_df))} tasks")
print("Breakdown By Category:\n", improved_by_hybrid_df['self_aware_failure'].value_counts()) 
#does that mean the agent becomes less aware of it's failures? it means it corrected it and then also becomes less aware 

Self-Refiner (Text) Improved 52 tasks
Breakdown By Category:
 self_aware_failure
Unaware    30
Aware      22
Name: count, dtype: int64

Self-Refiner (Vision) Improved 38 tasks
Breakdown By Category:
 self_aware_failure
Unaware    22
Aware      16
Name: count, dtype: int64

Self-Refiner (Hybrid) Improved 39 tasks
Breakdown By Category:
 self_aware_failure
Unaware    24
Aware      15
Name: count, dtype: int64


In [9]:
# Helper function to view the confusion matrix (if desired)
def print_confusion(gpt_eval):
    true_positive = gpt_eval[(gpt_eval['pred_score'] == 1) & (gpt_eval['score'] == 1)]
    true_negative = gpt_eval[(gpt_eval['pred_score'] == 0) & (gpt_eval['score'] == 0)]
    false_positive = gpt_eval[(gpt_eval['pred_score'] == 1) & (gpt_eval['score'] == 0)]
    false_negative = gpt_eval[(gpt_eval['pred_score'] == 0) & (gpt_eval['score'] == 1)]
    print(len(true_positive), len(true_negative))
    print(len(false_positive), len(false_negative))
    gpt_eval['validator_correct'] = gpt_eval['score'] == gpt_eval['pred_score']
    print("Accuracy:", gpt_eval['validator_correct'].mean())
    return

In [10]:
# # Get the subset of improved tasks with self-refinment (Correct workflows)
# improved_df_refine = self_refine_results[self_refine_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True).reset_index()
# improved_df_and_validation = self_refine_results[self_refine_results['validation_count'] > 1].sort_values(by='task_id', ascending=True).reset_index()

# # Get the subset of improved tasks from the original run (Incorrect workflows)
# improved_df_original = original_results[original_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True).reset_index()

## Analyzing Failure Cases with GPT-4

In [11]:
# Create directory to save outputs from failures cases
overwrite_files = 1
!mkdir -p error_analysis_outputs

In [12]:
# Get the subset of failure cases
original_wrong_df = original_results[original_results['score'] == 0].reset_index(drop=True)
refiner_wrong_df = self_refine_results[self_refine_results['score'] == 0].reset_index(drop=True)

In [13]:
print(f"{len(original_wrong_df)} of {len(original_results)} incorrect")
original_wrong_df[:2]

116 of 471 incorrect


,task_id,task_index,start_url,intent,last_url,tct,start_ts,completion_ts,last_statement,score,reason,compute_cost.cost,compute_cost.prompt_tokens,compute_cost.completion_tokens,compute_cost.total_tokens,total_main_chat_token,main_chat_length,total_trajectory_length
0,1,1,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna that ha...,https://www.allrecipes.com/recipe/78358/easy-v...,263.331766,2024-06-18 15:55:14,2024-06-18 15:59:38,After reviewing several recipes for vegetarian...,0.0,All these recipes do not explicitly include zu...,0.56686,53926.0,920.0,54846.0,1229,36,41
1,2,2,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna under 6...,https://www.allrecipes.com/recipe/278027/world...,184.683154,2024-06-18 16:18:27,2024-06-18 16:21:33,After reviewing the first three vegetarian las...,0.0,After reviewing the first three vegetarian las...,0.35565,32313.0,1084.0,33397.0,1375,22,27


In [14]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

print(f"{len(refiner_wrong_df)} of {len(self_refine_results)} incorrect")
refiner_wrong_df[:10][['intent', 'reason']]

91 of 471 incorrect


,intent,reason
0,Find a five-star rated chocolate chip cookie recipe that takes less than 1 hour to make on Allrecipes. Note how many reviews the recipe has and the main ingredients required.,not 5 stars
1,Find a vegan lasagna recipe on Allrecipes that requires 10 ingredients or less and has feedback of more than 200 reviews. Provide a brief overview of the ingredient list and the total prep and cook time.,no such recipe found
2,Find a recipe for a healthy avocado salad on Allrecipes that has a preparation time of less than 20 minutes and more than 30 user reviews. Include the nutritional information per serving.,chicekn salad shoqn
3,Find a popular quinoa salad recipe on Allrecipes with more than 500 reviews and a rating above 4 stars. Create a shopping list of ingredients for this recipe and include the total cooking and preparation time.,showing less than 500 reviws recipe
4,List 3 recommended dinner recipes in the Allrecipes Dinners section.,no 3 recipes listed
5,"Find the used Nintendo Switch Lite on Amazon then filter by 'Used - Good', tell me the cheapest one that is 'Used - Good'.",no filters applied in web page
6,"Find a pair of mens running shoes in black, size 7, 4+ stars and under $50 and add them to my cart on Amazon.",cart empty
7,Look for an English language book on roman empire history in the Amazon Kindle store. Sort by newests arrivals and look for a title that will be released within a month.,cudnt fetch results
8,"Find a stainless steel, 12-cup programmable coffee maker on Amazon. The price range should be between $100 to $200. Report the one with the 4+ customer rating.",cudnt find what was asked of
9,"Search for a queen-sized, hypoallergenic mattress topper on Amazon. It should have a memory foam material and be priced between $50 to $100.",failed to apply criterias


### Type 1: Self-Aware Failures

In [15]:
from openai import OpenAI
from tqdm import tqdm

# data = improved_df_original[['start_url', 'intent','reason']].to_csv()
# question = f"I need help analyzing my results from an experiment. In this experiment, I had a web-navigation agent accomplish many tasks on live websites. Each failure case was annotated by a real human with the reason for the web-navigator's failure. Help me analyze the failure cases. Please categorize the top-5 failure cases and give me examples of each. Think deeply about why each of these cases may occur and what causes each error type. Below is a csv file containing failure cases only. There are three columns: 1) start_url, which is the website the task was performed on 2) the intent, this is the task that the agent was given to perform and 3) the reason, this is the annotators' explaination for why the task failed: {data}"

In [16]:
def self_awareness_prompt_template(task, final_response):
    return f'''
You will be provided with the output of a web-navigation agent in response to a specific task. Your role is to determine if the agent is self-aware that it did not fully complete the task as instructed. Specifically, classify the agent's response as one of the following:

Self-Aware Failure: The agent explicitly acknowledges that it could not complete the task or identifies specific reasons for its failure (e.g., technical issues, access restrictions, incomplete information). 

Unaware Failure: The agent does not explicitly acknowledge its failure, even though the provided response does not fully address the task. 

Your answers should be in JSON format consisting of the "label" and "reasonsing". Label should be set to 1 if self-identified, otherwise set to 0. Below is an example of your task.

Example input: 
{{ "task" : "Find all Uniqlo locations in Chicago, IL.",
"response" : "Due to technical issues and access restrictions on the UNIQLO Store Locator page, we are unable to directly retrieve a comprehensive list of Uniqlo locations in Chicago, IL, at this time. However, based on the search results titles, it's indicated that there are multiple Uniqlo stores in Chicago, including locations at the Magnificent Mile, State Street, and possibly at Woodfield Mall. For the most accurate and up-to-date information, it is recommended to visit the official UNIQLO website or contact their customer service directly." }}

Example output:
{{
  "label": 1,
  "reasoning": "The agent explicitly acknowledges its inability to complete the task due to technical issues and access restrictions. It also provides alternative sources for accurate information, demonstrating self-awareness of its failure."
}}

Here is the task and response you need to label:
{{ "task" : "{task}",
"response" : "{final_response}"}}

Write your response below in JSON format:
'''

def call_self_awareness_judge(intent, final_response):
  self_awareness_prompt = self_awareness_prompt_template(intent, final_response)
  client = OpenAI()
  completion = client.chat.completions.create(
              model="gpt-4o",
              messages=[
                  {"role": "system", "content": "You are a helpful classification agent."},
                  {"role": "user", "content": self_awareness_prompt},
              ]
  )
  _message =  completion.choices[0].message.content
  clean_message = _message.replace("json", "").replace("\n", "").replace("```", "")
  return clean_message

def call_self_awareness_judge_for_dataframe(df, final_response_col_name="final_response"): #Note: the dataframe is assumed to have final_responses
  # Initialize new columns with None values
  df['self_aware'] = None
  df['self_aware_reason'] = None

  # Iterate over rows with iterrows
  for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
      # Call the function with appropriate arguments
      response = call_self_awareness_judge(row['intent'], row[final_response_col_name])
      
      # Parse the JSON response
      response_json = json.loads(response)
      
      # Assign values to the DataFrame
      df.loc[index, 'self_aware'] = response_json.get('label')
      df.loc[index, 'self_aware_reason'] = response_json.get('reasoning')  # Assuming the JSON has a 'reason' field

  return df

In [17]:
# Quick single test
index = 4
intent = original_wrong_df.loc[index, 'intent']
last_response = original_wrong_df.loc[index, 'last_statement']
message = call_self_awareness_judge(intent, last_response)
print(json.loads(message))

{'label': 0, 'reasoning': 'The agent provides a recipe that closely meets the criteria, although not explicitly stating it is five-star rated since it has a 4.6 rating. However, it gives the number of reviews, the main ingredients, and confirms the total time is under 1 hour. The agent does not acknowledge any failure or issue in completing the task, even though it technically did not find a five-star rated recipe.'}


In [18]:
# Full test
original_wrong_df = call_self_awareness_judge_for_dataframe(original_wrong_df, 
                                                               final_response_col_name="last_statement"
                                                               )
print(f"Self-Aware of {original_wrong_df['self_aware'].sum()/len(original_wrong_df)*100}% of failures improve cases")
original_wrong_df[['task_id', 'intent','last_statement', 'reason', 'self_aware', 'self_aware_reason']][:5]

Processing Rows:   0%|          | 0/116 [00:00<?, ?it/s]

Processing Rows:  22%|██▏       | 26/116 [00:47<02:43,  1.82s/it]


KeyboardInterrupt: 

In [ ]:
# Save your results
# Warning: This might overwrite prior runs!
original_wrong_df.to_csv('error_analysis_outputs/original_annotations.csv', index=False)

In [ ]:
# # Full test
refiner_wrong_df_by_planner = call_self_awareness_judge_for_dataframe(refiner_wrong_df, 
                                                               final_response_col_name="last_statement"
                                                               )
print(f"Self-Aware of {refiner_wrong_df_by_planner['self_aware'].sum()/len(refiner_wrong_df_by_planner)*100}% of failures in self-refinement according to the last planner call")
refiner_wrong_df_by_planner[['task_id', 'intent','last_statement', 'validator_last_statement', 'reason', 'self_aware', 'self_aware_reason']][:2]

Processing Rows:   1%|          | 1/91 [00:01<01:43,  1.15s/it]

In [ ]:
validator_identified_failure_count = refiner_wrong_df['validator_last_statement'].str.contains('The workflow was not successfully completed', na=False).sum()
print(f"Self-Aware of {(validator_identified_failure_count/len(refiner_wrong_df)*100):0.2f}% of failures according to the last validator")

Self-Aware of 0.00% of failures according to the last validator


In [ ]:
print(f"Agent-E No-Refinement: Self-Aware of {original_wrong_df['self_aware'].sum()/len(original_wrong_df)*100:0.2f}% ")
print(f"Agent-E Refinment (Text): Self-Aware of {(validator_identified_failure_count/len(refiner_wrong_df)*100):0.2f}% of failures according to the last validator")

improved_df_original = original_wrong_df[original_wrong_df['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True).reset_index()

print(f"Agent-E No-Refinement Improved Cases: Self-Aware of {improved_df_original['self_aware'].sum()/len(improved_df_original)*100:0.2f}% ")
improved_df_original[['task_id', 'intent','last_statement', 'reason', 'self_aware', 'self_aware_reason']][:5]

Agent-E No-Refinement: Self-Aware of 49.14% 
Agent-E Refinment (Text): Self-Aware of 0.00% of failures according to the last validator


NameError: name 'improved_tasks' is not defined

In [ ]:
len(improved_df_original)

64

In [ ]:
# # Save your results
# # Warning: This might overwrite prior runs!
# original_wrong_df.to_csv('error_analysis_outputs/original_annotations.csv', index=False)

### Type 2: No response

In [ ]:
def categorize_error_prompt_template(task, annotator_reason):
    return f'''
You will be provided with the output of a web-navigation agent in response to a specific task and other information about a failured workflow. Your role is to categories the cause of failure. Specifically, classify the agent's response as one of the following:

No Response: In this case, the web-agent failured the task because it either left the response empty or did the task but didn't fully answer the question. This type of failure should be identified by using the annotator_reason. If the annotator indicated that the agent had no response or did not answer the originally intended question, then it should be labeled as this type of failure.

Navigation: In this case, the agent failes to perform the given task due to navigation related issue

Widget Limitation: In this case, the agent failes to perform a given task, because the agent is not capable of use the appropriate filters (e.g. widgets)

Other: Any other type of failure should be labeled as "Other"


Your answers should be in JSON format consisting of the "label" and "reasonsing":

Example input: 
{{ "task" : "Search for women's golf polos in m size, priced between 50 to 75 dollars, and save the lowest priced among results.",
"annotator_reason" : "blank answer" }}

Example output 1:
{{
  "label": "No Response",
  "reasoning": "The annotator indicated that the agent answer was left blank."
}}

Here is the task and response you need to label:
{{ "task" : "{task}",
"annotator_reason" : "{annotator_reason}"}}

Write your response below in JSON format:
'''

def call_categorize_error_judge(intent, annotator_reason):
  categorize_error_prompt = categorize_error_prompt_template(intent, annotator_reason)
  client = OpenAI()
  completion = client.chat.completions.create(
              model="gpt-4o",
              messages=[
                  {"role": "system", "content": "You are a helpful classification agent."},
                  {"role": "user", "content": categorize_error_prompt},
              ]
  )
  _message =  completion.choices[0].message.content
  clean_message = _message.replace("json", "").replace("\n", "").replace("```", "")
  return clean_message

def call_categorize_error_judge_for_dataframe(df, annotator_reason_col_name="reason"): #Note: the dataframe is assumed to have final_responses
  # Initialize new columns with None values
  df['error_type'] = None
  df['error_type_reason'] = None

  # Iterate over rows with iterrows
  for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
      # Call the function with appropriate arguments
      response = call_categorize_error_judge(row['intent'], annotator_reason=row[annotator_reason_col_name])
      
      # Parse the JSON response
      print(response)
      response_json = json.loads(response)
      
      # Assign values to the DataFrame
      df.loc[index, 'error_type'] = response_json.get('label')
      df.loc[index, 'error_type_reason'] = response_json.get('reasoning')  # Assuming the JSON has a 'reason' field
  return df

In [ ]:
# Quick single test
index = 6
intent = refiner_wrong_df.loc[index, 'intent']
last_response = refiner_wrong_df.loc[index, 'reason']
message = call_categorize_error_judge(intent, last_response)
print(intent)
print(json.loads(message), "\n")

Search for women's golf polos in m size, priced between 50 to 75 dollars, and save the lowest priced among results.
{'label': 'No Response', 'reasoning': 'The annotator indicated that the agent answer was left blank.'}


In [ ]:
# # Full test
refiner_wrong_df_by_planner = call_categorize_error_judge_for_dataframe(refiner_wrong_df[:10], 
                                                               annotator_reason_col_name="reason"
                                                               )
#print(f"Self-Aware of {refiner_wrong_df_by_planner['self_aware'].sum()/len(refiner_wrong_df_by_planner)*100}% of failures in self-refinement according to the last planner call")
refiner_wrong_df_by_planner[['task_id', 'intent','last_statement', 'validator_last_statement', 'reason', 'error_type', 'error_type_reason']]

/var/folders/57/sn2wbp7562s9f9f6yfq33ty40000gn/T/ipykernel_36527/2246996306.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_type'] = None
/var/folders/57/sn2wbp7562s9f9f6yfq33ty40000gn/T/ipykernel_36527/2246996306.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_type_reason'] = None
Processing Rows:  10%|█         | 1/10 [00:02<00:20,  2.31s/it]

{  "label": "Other",  "reasoning": "The annotator indicated that the issue was related to the agent misunderstanding the difference between reviews and ratings, not that the response was empty or left blank."}


Processing Rows:  20%|██        | 2/10 [00:04<00:18,  2.30s/it]

{  "label": "Other",  "reasoning": "The agent misinterpreted the task by not adhering to the specified constraint of preparation time being under 30 minutes, as indicated by the annotator."}


Processing Rows:  30%|███       | 3/10 [00:07<00:18,  2.67s/it]

...

Processing Rows:  90%|█████████ | 9/10 [00:23<00:02,  2.48s/it]

{  "label": "Other",  "reasoning": "The annotator indicated that the failure was due to the price filter not being applied. This implies an incorrect execution of the task rather than a lack of response."}


Processing Rows: 100%|██████████| 10/10 [00:26<00:00,  2.67s/it]

{  "label": "Other",  "reasoning": "The annotator provided specific information about the starting price of the pro max model, indicating that the agent attempted to perform the task but may not have fully completed the price and screen size comparison. However, this does not fit the criteria for a 'No Response' classification as there is a response present."}


,task_id,intent,last_statement,validator_last_statement,reason,error_type,error_type_reason
0,6,Search for a vegetarian lasagna recipe that has at least a four-star rating and over 500 reviews.,"The vegetarian lasagna recipe that meets your criteria is 'Spinach Lasagna,' with a rating of 4.7 out of 5 based on 733 reviews. You can view the recipe at [https://www.allrecipes.com/recipe/22729/spinach-lasagna-iii/](https://www.allrecipes.com/recipe/22729/spinach-lasagna-iii/).","The task was completed. The workflow was successfully completed as all parts of the task were executed correctly. The user started from the AllRecipes homepage, searched for 'vegetarian lasagna' recipes, confirmed the search results page, manually found a recipe that met the criteria, and extracted the necessary details from the recipe page.",733 are not reviews those are ratings. 505 ratings,Other,"The annotator indicated that the issue was related to the agent misunderstanding the difference between reviews and ratings, not that the response was empty or left blank."
1,14,"Search for a recipe that includes ""chicken breast"" and ""quinoa"" with preparation time under 30 minutes on Allrecipes.",I found a recipe that meets your criteria: 'Chicken with Quinoa and Veggies'. The preparation time is 30 minutes. You can view the recipe at https://www.allrecipes.com/recipe/215076/chicken-with-quinoa-and-veggies/.,"The task was completed. The workflow was successfully completed as the user was able to find a recipe that includes 'chicken breast' and 'quinoa' with a preparation time of 30 minutes, which meets the criteria of being under 30 minutes. The user started on the Allrecipes homepage, performed the search, confirmed the search results, and manually checked the preparation time of the first recipe, which met the criteria. The recipe details were extracted successfully.",asked for prep time under 30 not equals 30 min,Other,"The agent misinterpreted the task by not adhering to the specified constraint of preparation time being under 30 minutes, as indicated by the annotator."
2,16,Find a five-star rated chocolate chip cookie recipe that takes less than 1 hour to make on Allrecipes. Note how many reviews the recipe has and the main ingredients required.,"The five-star rated chocolate chip cookie recipe that takes less than 1 hour to make on Allrecipes is 'Best Chocolate Chip Cookies'. It has 14,581 reviews. The main ingredients required are:- 1 cup butter, softened- 1 cup white sugar- 1 cup packed brown sugar- 2 eggs- 2 teaspoons vanilla extract- 1 teaspoon baking soda- 2 teaspoons hot water- ½ teaspoon salt- 3 cups all-purpose flour- 2 cups semisweet chocolate chips- 1 cup chopped walnuts (optional)These ingredients are used to create cookies with crisp edges and chewy middles.",The workflow was successfully completed as all parts of the task were addressed. A five-star rated chocolate chip cookie recipe that takes less than 1 hour to make was found on Allrecipes. The number of reviews and the main ingredients required for the recipe were also noted and listed.,4.6 stars,Other,The agent provided a recipe with 4.6 stars instead of fulfilling the task requirement of finding a five-star rated recipe. This indicates a misunderstanding or misexecution of the task criteria rather than an empty or incomplete response.
3,22,Find a recipe for a healthy avocado salad on Allrecipes that has a preparation time of less than 20 minutes and more than 30 user reviews. Include the nutritional information per serving.,"I found a recipe that meets your criteria on Allrecipes. The 'Taco Salad with Lime Vinegar Dressing' has a preparation time of 20 minutes and more than 30 user reviews. The nutritional information per serving is 554 calories, 26g of fat, 52g of carbs, and 31g of protein. You can view the recipe here: [Taco Salad with Lime Vinegar Dressing Recipe](https://www.allrecipes.com/recipe/241982/taco-salad-with-lime-vinegar-dressing/).","The workflow was not successfully 

### Peaking at improved cases

In [ ]:
improved_df_original = original_results[original_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True).reset_index()

In [ ]:
improved_df_original[['task_id', 'intent', 'score', 'reason', 'last_statement']]

,task_id,intent,score,reason,last_statement
0,1,Find a recipe for a vegetarian lasagna that has at least a four-star rating and uses zucchini.,0.0,All these recipes do not explicitly include zucchini in their ingredients list.,"After reviewing several recipes for vegetarian lasagna on AllRecipes, none of the top-rated recipes explicitly include zucchini in their ingredients list. While some reviews suggest modifications that include zucchini, no original recipe from the selection meets all the specified criteria of being vegetarian, having at least a four-star rating, and using zucchini."
1,2,Find a recipe for a vegetarian lasagna under 600 calories per serving that has a prep time of less than 1 hour.,0.0,"After reviewing the first three vegetarian lasagna recipes from Allrecipes, none meet the specific criteria of being under 600 calories per serving with a prep time of less than 1 hour.","After reviewing the first three vegetarian lasagna recipes from Allrecipes, none meet the specific criteria of being under 600 calories per serving with a prep time of less than 1 hour. The recipes explored either exceeded the total time requirement or did not explicitly match all the specified criteria."
2,8,Search for a recipe for Beef Wellington on Allrecipes that has at least 200 reviews and an average rating of 4.5 stars or higher. List the main ingredients required for the dish.,0.0,none of the recipes meet the specified criteria of having at least 200 reviews and an average rating of 4.5 stars or higher.,"After reviewing the available Beef Wellington recipes on Allrecipes, none of the recipes meet the specified criteria of having at least 200 reviews and an average rating of 4.5 stars or higher. The closest match was 'Individual Beef Wellingtons' with 189 ratings and an average rating of 4.7 stars, but it falls short of the required number of reviews. There are no other recipes in the search results that fulfill both criteria."
3,18,Find a recipe for a vegetarian lasagna that has over 300 reviews and an average rating of 4.5 or higher on Allrecipes.,0.0,The page does not directly address the initial task of finding a vegetarian lasagna recipe with over 300 reviews and an average rating of 4.5 or higher on Allrecipes.,"It appears there has been a misunderstanding in the navigation process, leading to a page that does not directly address the initial task of finding a vegetarian lasagna recipe with over 300 reviews and an average rating of 4.5 or higher on Allrecipes. Due to the limitations encountered in accessing and verifying specific recipe details through the provided DOM and navigation issues, I'm unable to complete the task as initially requested."
4,25,Find a popular quinoa salad recipe on Allrecipes with more than 500 reviews and a rating above 4 stars. Create a shopping list of ingredients for this recipe and include the total cooking and preparation time.,0.0,The search result doesn't meet the requirement of 500 reviews and it only has 441 reviews.,"The 'Mediterranean Quinoa Salad' recipe from Allrecipes meets your criteria with a rating of 4.8 out of 5 based on 604 ratings. Here are the details:\n\n**Total Cooking and Preparation Time:** 30 minutes (15 minutes prep time, 15 minutes cook time)\n\n**Shopping List of Ingredients:**\n- 2 cups water\n- 2 cubes chicken bouillon\n- 1 clove garlic, smashed\n- 1 cup uncooked quinoa\n- 2 large cooked chicken breasts - cut into bite-size pieces (Optional)\n- 1 large red onion, diced\n- 1 large green bell pepper, diced\n- ½ cup chopped kalamata olives\n- ½ cup crumbled feta cheese\n- ¼ cup chopped fresh parsley\n- ¼ cup chopped fresh chives\n- ½ teaspoon salt\n- ⅔ cup fresh lemon juice\n- 1 tablespoon balsamic vinegar\n- ¼ cup olive oil\n\nThis recipe is versatile and can be served warm or cold, making it a delightful dish for any occasion."
5,30,"Find a recipe for a vegan smoothie bowl on Allrecipes that includes bananas and leaves, has more than 20 reviews, and a rating of at least 

## Failure breakdown (continued ...)

In [ ]:
improved_df_original = original_results[original_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True).reset_index()

In [ ]:
from validation_agent.analyze_failure import *

model = "gpt-4o"
task_id = list(improved_df_original['task_id'])[0]
main_chat_sequence = original_annotation.get_high_level_trajectory(task_id)
screenshot_paths = original_annotation.get_screenshot_paths(task_id)
intent = main_chat_sequence[0].get("content", "")

analysis = analyze_failure_text(main_chat_sequence, intent, model=model)

Model is being used: gpt-4o


In [ ]:
# Store all of them into a dataframe (i.e. loop from improved_tasks)
# Then summarize from a high-level (i.e. call openai for another step of anlaysis)
import json

response = analysis.get('pred_raw_response', None).replace("json", "").replace("```", "")
print(response)
response = json.loads(response)
rationale = response.get("rationale")
improvement = response.get("improvement")

for task_id in improved_df_original['task_id']:
        try:
            # Get data for the current task_id
            main_chat_sequence = original_annotation.get_high_level_trajectory(task_id)
            screenshot_paths = original_annotation.get_screenshot_paths(task_id)
            intent = main_chat_sequence[0].get("content", "")

            # Run the analysis function
            analysis_result = analyze_failure_text(main_chat_sequence, intent, model=model)

            improved_df_original[]

SyntaxError: invalid syntax (1132630562.py, line 21)

In [ ]:
# TODO: go back and add ALL organized data...

'Execute the user task "Find a recipe for a vegetarian lasagna that has at least a four-star rating and uses zucchini." Current Page: https://www.allrecipes.com/'

TODO TO ANALYZE IMPROVEMENTS:
1. figure out cases of errors (i.e. identify examples)
    Type A: Bad navigation <- most popular and easy to identify
    Type B: Failure to use widgets 
    Type C: Doesn't fully answer the agent's questions
2. 